# Duck Typing, Interfaces, and Benchmarking

Julia's 'interfaces' and duck-typing are a core part of Julia's type system.

> A lot of the power and extensibility in Julia comes from a collection of informal interfaces. By extending a few specific methods to work for a custom type, objects of that type not only receive those functionalities, but they are also able to be used in other methods that are written to generically build upon those behaviors.

## Example 1: `UnitRange`

In [1]:
x = 1:30

1:30

In [2]:
typeof(x)

UnitRange{Int64}

In [3]:
supertypes(UnitRange)

(UnitRange, AbstractUnitRange{T} where T<:Real, OrdinalRange{T, T} where T<:Real, AbstractRange{T} where T<:Real, AbstractVector{T} where T<:Real, Any)

In [4]:
typeof(x) <: AbstractArray

true

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [5]:
x[3]

3

In [6]:
size(x)

(30,)

In [7]:
eltype(x)

Int64

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [8]:
fieldnames(typeof(x))

(:start, :stop)

or just by inspecting the source code

In [9]:
@which UnitRange{Int64}(1, 30)

UnitRange{T}(start::T, stop::T) where T<:Real in Base at range.jl:393

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Allocating memory is typically costly.

In [11]:
@time collect(1:10_000_000);

  0.051380 seconds (2 allocations: 76.294 MiB, 19.67% gc time)


But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [12]:
@time 1:10_000_000;

  0.000001 seconds


This is so far the same as in Python, but here is a key difference:

```ipython
In [1]: a = range(10000)

In [2]: a
Out[2]: range(0, 10000)

In [3]: a * 2
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[3], line 1
----> 1 a * 2

TypeError: unsupported operand type(s) for *: 'range' and 'int'
```

As Julia uses multiple dispatch, `UnitRange` has its own methods defined for arithmetic which are **custom made for that type**. Not only does it behave the same way as an array with the same operations, it also does so **with great performance**:

In [16]:
# Double all elements in the range and print off the 10th element:
@time r = collect(1:10_000_000) * 2
@time r = (1:10_000_000) * 2
println(r[10])

  0.130613 seconds (4 allocations: 152.588 MiB, 67.14% gc time)
  0.000002 seconds
20


In [19]:
@which sum(1:10_000_000)

sum(r::AbstractRange{<:Real}) in Base at range.jl:1387

In [18]:
# Sum all of the elements in the range:
@time sum(collect(1:10_000_000))
@time sum(1:10_000_000)

  0.054914 seconds (2 allocations: 76.294 MiB, 6.92% gc time)
  0.000001 seconds


50000005000000

Parts of this do work in Python:

```ipython
In [1]: %time sum(range(10_000_000))
CPU times: user 183 ms, sys: 169 µs, total: 183 ms
Wall time: 183 ms
```

In [20]:
using BenchmarkTools  # This will be explained later
using PythonCall  # As well as this

@btime @pyexec "sum(range(10_000_000))"

    CondaPkg Found dependencies: /home/roscar/work/github.com/RobertRosca/julia-hpc-workshop/CondaPkg.toml
    CondaPkg Found dependencies: /home/roscar/.local/share/julia/packages/PythonCall/3GRYN/CondaPkg.toml
    CondaPkg Resolving changes
             + libstdcxx-ng
             + numpy
             + python
    CondaPkg Installing packages
             │ /home/roscar/.local/share/julia/artifacts/134b3f7254989e369343bab626bc7e0c77a9cc1a/bin/micromamba
             │ -r /home/roscar/.local/share/julia/scratchspaces/0b3b1443-0f03-428d-bdfb-f27f9c1191ea/root
             │ create
             │ -y
             │ -p /home/roscar/work/github.com/RobertRosca/julia-hpc-workshop/.CondaPkg/env
             │ --override-channels
             │ --no-channel-priority
             │ libstdcxx-ng[version='>=3.4,<=12.2',channel='conda-forge']
             │ numpy[version='*']
             │ python[version='>=3.7,<4',channel='conda-forge',build='*cpython*']
             └ -c conda-forge



                                           __
          __  ______ ___  ____ _____ ___  / /_  ____ _
         / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
        / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
       / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
      /_/


Transaction

  Prefix: /home/roscar/work/github.com/RobertRosca/julia-hpc-workshop/.CondaPkg/env

  Updating specs:

   - conda-forge::libstdcxx-ng[version='>=3.4,<=12.2']
   - numpy=*
   - conda-forge::python[version='>=3.7,<4',build=*cpython*]


  Package               Version  Build                Channel                    Size
───────────────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  conda_forge          conda-forge/linux-64     Cached
  + _openmp_mutex           4.5  2_gnu                conda-forge/linux-64     Cached
  + bzip2                 1.0.

LoadError: InterruptException:

This effectively does the same thing the `collect` example does, just a few times slower, and far slower than the example using a range.

In [21]:
@which sum(1:10_000_000)

sum(r::AbstractRange{<:Real}) in Base at range.jl:1387

Duck typing and interfaces let us define specialised code that is very performant, and interacts with the rest of the language as expected.

## Example 2: `DiagonalMatrix`

Let's create a simple custom `DiagonalMatrix` type that can represent square diagonal matrices, i.e.

$$ D = \left( \begin{matrix} x & 0 & 0 & 0 \\ 0 & y & 0 & 0 \\ 0 & 0 & z & 0 \\ 0 & 0 & 0 & \ddots \end{matrix} \right) $$

In [22]:
struct DiagonalMatrix{T} <: AbstractArray{T, 2}
    diag::Vector{T}
end

In the spirit of duck typing, we integrate our `DiagonalMatrix` into Julia's type hierarchy by making it a subtype (`<:`) of `AbstractMatrix` to indicate **array-like behavior**. (Note that this does not indicate inheritence of structure!)

Of course, to actually make it behave like a matrix (a two-dimensional array) we must also implement (parts of) the [`AbstractArray` interface](https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1).

In [23]:
DiagonalMatrix([1, 2, 3])

MethodError: MethodError: no method matching size(::DiagonalMatrix{Int64})
[0mClosest candidates are:
[0m  size(::AbstractArray{T, N}, [91m::Any[39m) where {T, N} at abstractarray.jl:42
[0m  size([91m::Union{LinearAlgebra.Adjoint{T, var"#s886"}, LinearAlgebra.Transpose{T, var"#s886"}} where {T, var"#s886"<:(AbstractVector)}[39m) at /usr/share/julia/stdlib/v1.8/LinearAlgebra/src/adjtrans.jl:173
[0m  size([91m::Union{LinearAlgebra.Adjoint{T, var"#s886"}, LinearAlgebra.Transpose{T, var"#s886"}} where {T, var"#s886"<:(AbstractMatrix)}[39m) at /usr/share/julia/stdlib/v1.8/LinearAlgebra/src/adjtrans.jl:174
[0m  ...

In [24]:
Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

In [25]:
DiagonalMatrix([1, 2, 3])

CanonicalIndexError: CanonicalIndexError: getindex not defined for DiagonalMatrix{Int64}

In [26]:
function Base.getindex(D::DiagonalMatrix, i::Int, j::Int)
    if i == j
        return D.diag[i]
    else
        return zero(eltype(D))
    end
end

In [27]:
D = DiagonalMatrix([1, 2, 3])

3×3 DiagonalMatrix{Int64}:
 1  0  0
 0  2  0
 0  0  3

Note how it's automagically pretty printed (despite the fact that we never defined any special printing)!

In [28]:
D[2, 2]

2

In [29]:
D[1, 2]

0

In [30]:
size(D)

(3, 3)

In [31]:
D[3, 3] = 5

LoadError: CanonicalIndexError: setindex! not defined for DiagonalMatrix{Int64}

In [32]:
function Base.setindex!(D::DiagonalMatrix, v, i::Int, j::Int)
    if i == j
        D.diag[i] = v
    else
        throw(ArgumentError("cannot set off-diagonal entry ($i, $j)"))
    end
    return v
end

In [33]:
D[3, 3] = 5

5

In [34]:
D

3×3 DiagonalMatrix{Int64}:
 1  0  0
 0  2  0
 0  0  5

In [35]:
D[3, 4] = 5

LoadError: ArgumentError: cannot set off-diagonal entry (3, 4)

But that's not it. Because of duck typing, all kinds of different functions now "just work".

In [36]:
eltype(D) # element data type

Int64

In [37]:
D + D # addition

3×3 Matrix{Int64}:
 2  0   0
 0  4   0
 0  0  10

In [38]:
D * D # multiplication

3×3 Matrix{Int64}:
 1  0   0
 0  4   0
 0  0  25

In [39]:
inv(D) # inversion

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  0.5  0.0
 0.0  0.0  0.2

In [40]:
sin.(D) # broadcasting

3×3 Matrix{Float64}:
 0.841471  0.0        0.0
 0.0       0.909297   0.0
 0.0       0.0       -0.958924

In [ ]:
using LinearAlgebra
eigen(D) # eigensolver

Of course, so far, these operations have suboptimal performance because they don't utilize the special structure inherent to our `DiagonalMatrix` but fall back to generic implementations.

In [41]:
@which D + D

+(A::AbstractArray, B::AbstractArray) in Base at arraymath.jl:6

In [42]:
Base.:+(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag + Db.diag)

In [43]:
@which D + D

+(Da::DiagonalMatrix, Db::DiagonalMatrix) in Main at In[42]:1

Important note: **user defined types are just as good as built-in types!**

There is nothing special about built-in types. In fact, [they are implemented in essentially the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Whereas diagonal arrays in something like Numpy are [some C code](https://github.com/numpy/numpy/blob/d4b2d4f80060285ac085ea874aceaf9fa1bfb757/numpy/core/src/multiarray/item_selection.c#L1964)

## Side Note: Consistency of Interfaces

Another benefit of the abstract type/interface/multiple dispatch style is that interfaces are very consistent within Julia, across not just the base language but across packages as well.

An article called [Python vs. Julia: It's also about Consistency](https://towardsdatascience.com/python-vs-julia-its-also-about-consistency-236812dd64ba) shows this point quite well.

## Benchmarking with [`BenchmarkTools.jl`](https://github.com/JuliaCI/BenchmarkTools.jl)

Benchmarking is difficult to do right for many reasons
* computers are noisy machines
* global vs local scope
* the first function call is special in Julia (more later)
* ...

In [52]:
g(x) = x + 2 * x

g (generic function with 1 method)

In [53]:
x = rand(2, 2)
@time g.(x)

  0.063475 seconds (218.67 k allocations: 11.180 MiB, 99.34% compilation time)


2×2 Matrix{Float64}:
 0.546315  1.89182
 2.57299   0.017898

In [ ]:
function f()
    x = rand(2, 2)
    @time g.(x)
end

In [ ]:
f()

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

General rule: **Don't use `@time` but `@btime`** from [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) and **interpolate (`$`) input arguments**.

In [58]:
using BenchmarkTools

In [61]:
@btime g.(x)

  439.313 ns (4 allocations: 160 bytes)


2×2 Matrix{Float64}:
 0.546315  1.89182
 2.57299   0.017898

In [62]:
@btime g.($x)

  41.057 ns (1 allocation: 96 bytes)


2×2 Matrix{Float64}:
 0.546315  1.89182
 2.57299   0.017898

In [60]:
@benchmark g.($x)

BenchmarkTools.Trial: 10000 samples with 991 evaluations.
 Range (min … max):  39.206 ns …  2.546 μs  ┊ GC (min … max): 0.00% … 95.60%
 Time  (median):     42.290 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   48.935 ns ± 69.285 ns  ┊ GC (mean ± σ):  5.99% ±  4.24%

  ▃▅█▆▄▄▃▄▆▅▃▃▂▂▂▁▁      ▁       ▁▁                           ▂
  ██████████████████████████▇▇███████▇██▇▇▆▅▅▅▅▅▅▅▄▄▄▅▅▄▅▄▄▄▄ █
  39.2 ns      Histogram: log(frequency) by time      86.3 ns <

 Memory estimate: 96 bytes, allocs estimate: 1.

### Custom types are just as good as built-in types

Let's compare our custom `DiagonalMatrix` against the standard `Diagonal` type that ships in the `LinearAlgebra` standard library.

In [63]:
using LinearAlgebra

x = rand(100);

Djl = Diagonal(x)
D = DiagonalMatrix(x)

@btime $Djl + $Djl;
@btime $D + $D;

  68.156 ns (1 allocation: 896 bytes)
  68.263 ns (1 allocation: 896 bytes)


## Example 3: Cooler Diagonal Matrix

What we made works quite well, but could be even neater if we remove the `Vector` restriction, and instead just use `AbstractVector`s. That way it could even work with ranges! 

In [64]:
struct CoolerDiagonalMatrix{T} <: AbstractArray{T, 2}
    diag::AbstractVector{T}
end

function Base.getindex(D::CoolerDiagonalMatrix, i::Int, j::Int)
    if i == j
        return D.diag[i]
    else
        return zero(eltype(D))
    end
end

Base.:+(Da::CoolerDiagonalMatrix, Db::CoolerDiagonalMatrix) = CoolerDiagonalMatrix(Da.diag + Db.diag)

In [66]:
# Old version:
D = DiagonalMatrix(collect(1:10_000))
@btime $D + $D

# New:
D_cool = CoolerDiagonalMatrix(1:10_000)
@btime $D_cool + $D_cool;

  3.982 μs (2 allocations: 78.17 KiB)
  109.001 ns (3 allocations: 112 bytes)


Works! But... why are the allocations up? Let's explore more:

In [68]:
D = DiagonalMatrix(collect(1:1000))
@btime $D + $D

D_cool = CoolerDiagonalMatrix(collect(1:1000))
@btime $D_cool + $D_cool;

  639.912 ns (1 allocation: 7.94 KiB)
  789.414 ns (2 allocations: 7.95 KiB)


Huh... the new type is slower even with the same arguments? Why?

We'll explore this in the sessions tomorrow.

p.s. Numpy:

```ipython
In [1]: import numpy as np

In [2]: d = np.diag(np.arange(1000))

In [3]: %timeit d + d
2.56 µs ± 35.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
```

Is still slower than any of the Julia code. 

In [ ]:
using PythonCall

@btime @pyexec """
import numpy as np

d = np.diag(np.arange(1_000))

d + d
"""

# Core messages of this notebook

* Duck typing is about **shared behavior** instead of shared structure.
* **User defined types are as good as built-in types.**
* We can **extend Base functions** for our types to implement arithmetics and such.
* **Subtyping an existing interface** can give lots of functionality for free.
* Functions should almost always be benchmarked with **BenchmarkTools.jl's `@btime` and `@benchmark`** instead of `@time`.

# References

- https://github.com/carstenbauer/JuliaHLRS22/blob/main/Day1/2_duck_typing_and_benchmarking.ipynb
- https://towardsdatascience.com/python-vs-julia-its-also-about-consistency-236812dd64ba
- https://docs.julialang.org/en/v1/manual/performance-tips/#Avoid-untyped-global-variables